### Ziel dieses Notebooks ist es unseren bestehenden Datensatz zu erweitern, um das Feature CreatorID. Hierfür haben wir 58 Kickstarter CSV Dateien von einer anderen Quelle heruntergeladen. Diese CSV Dateien möchten wir zunächst zusammenfügen und dann anhand der Project ID unseren bestehenden Datensatz mit dem neuen mergen.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import dirname
import datetime as dt

notebook_path = os.path.abspath("DataAnalyticsKickstarterNotebook_Cedrik.ipynb")
csv_path = os.path.join(os.path.dirname(dirname(notebook_path)), "data/Kickstarter_2020-12-17/Kickstarter000.csv")

In [4]:
# Ausgabe einer neuen CSV Datei
df = pd.read_csv (csv_path, low_memory=False)
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,1,Bringing deliciously fresh snacking to every g...,"{""id"":311,""name"":""Food Trucks"",""slug"":""food/fo...",50,US,the United States,1450855441,"{""id"":1480037339,""name"":""Palmetto Pork Skins (...",USD,$,...,palmetto-pork-skins,https://www.kickstarter.com/discover/categorie...,False,False,failed,1453922267,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",50.000000,domestic
1,610,Warm hands and full access to your camera dial...,"{""id"":333,""name"":""Camera Equipment"",""slug"":""te...",40990,NO,Norway,1448279489,"{""id"":1823457299,""name"":""Vallerret Photography...",NOK,kr,...,photography-gloves-extend-your-session-in-style,https://www.kickstarter.com/discover/categorie...,True,False,successful,1451336400,0.115882,"{""web"":{""project"":""https://www.kickstarter.com...",41198.350864,domestic
2,11,Enamel pins dedicated to my Grandpa and his di...,"{""id"":262,""name"":""Accessories"",""slug"":""fashion...",253,US,the United States,1571882707,"{""id"":1195320399,""name"":""Enzo Daniel"",""slug"":""...",USD,$,...,eat-shit-a-memorial-pin-campaign,https://www.kickstarter.com/discover/categorie...,True,False,successful,1575075016,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",253.000000,domestic
3,50,"To Rise In Challenge is a brand, lifestyle, me...","{""id"":263,""name"":""Apparel"",""slug"":""fashion/app...",4475,US,the United States,1586697917,"{""id"":1752361004,""name"":""Brandon Maddux"",""slug...",USD,$,...,tric-clothing-co,https://www.kickstarter.com/discover/categorie...,True,False,successful,1604171238,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",4475.000000,domestic
4,442,Inspiring girls & young women. Join me as I cr...,"{""id"":27,""name"":""Graphic Design"",""slug"":""desig...",38821,AU,Australia,1468395078,"{""id"":1671102323,""name"":""Wendy Fox"",""slug"":""wo...",AUD,$,...,womens-gold-medalists-rio-2016,https://www.kickstarter.com/discover/categorie...,True,True,successful,1472162364,0.746784,"{""web"":{""project"":""https://www.kickstarter.com...",38010.566960,domestic


In [5]:
# Die 58 verschiedenen CSV Datein zunächst in ein Dataframe laden
for i in range(58):
    if i <= 8:
        csv_path_new = os.path.join(os.path.dirname(dirname(notebook_path)), f"data/Kickstarter_2020-12-17/Kickstarter00{i+1}.csv")
    else: 
        csv_path_new = os.path.join(os.path.dirname(dirname(notebook_path)), f"data/Kickstarter_2020-12-17/Kickstarter0{i+1}.csv")
    
    df_new = pd.read_csv (csv_path_new, low_memory=False)
    df = df.append(df_new)

In [6]:
# Bereits hier ist schon erkennbar, dass der neue Datensatz weniger Einträge hat
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217253 entries, 0 to 3149
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             217253 non-null  int64  
 1   blurb                     217245 non-null  object 
 2   category                  217253 non-null  object 
 3   converted_pledged_amount  217253 non-null  int64  
 4   country                   217253 non-null  object 
 5   country_displayable_name  217253 non-null  object 
 6   created_at                217253 non-null  int64  
 7   creator                   217253 non-null  object 
 8   currency                  217253 non-null  object 
 9   currency_symbol           217253 non-null  object 
 10  currency_trailing_code    217253 non-null  bool   
 11  current_currency          217253 non-null  object 
 12  deadline                  217253 non-null  int64  
 13  disable_communication     217253 non-null  boo

In [16]:
df["blurb"][df["id"] == 480501031]

3278    A month-long pop-up workshop & design studio i...
Name: blurb, dtype: object

In [8]:
# Auswahl der relevanten Spaltern creator und id
df_cleaned = df[["creator","id"]]    
df_cleaned.head()

,creator,id
0,"{""id"":1480037339,""name"":""Palmetto Pork Skins (...",1124922936
1,"{""id"":1823457299,""name"":""Vallerret Photography...",1479488174
2,"{""id"":1195320399,""name"":""Enzo Daniel"",""slug"":""...",2077771673
3,"{""id"":1752361004,""name"":""Brandon Maddux"",""slug...",2008689997
4,"{""id"":1671102323,""name"":""Wendy Fox"",""slug"":""wo...",1113925398


In [9]:
# Slicen der creator id und erstellen einer neuen spalte 
string_list = df_cleaned['creator'].str.split(",", expand=True)
string_list = string_list[0].str.split(":",expand=True)
string_list[1]
df_creator_id = df_cleaned
df_creator_id["creator_id"] = string_list[1].astype('int64')
df_creator_id = df_creator_id.drop(columns="creator")

C:\Users\Cedrik\AppData\Local\Temp/ipykernel_16720/1186518534.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_creator_id["creator_id"] = string_list[1].astype('int64')


In [11]:
df_creator_id = df_creator_id.rename(columns={"id":"ID"})
df_creator_id.head()

,ID,creator_id
0,1124922936,1480037339
1,1479488174,1823457299
2,2077771673,1195320399
3,2008689997,1752361004
4,1113925398,1671102323


In [13]:
# Doppelte Projekte entfernen, damit es beim Merge zu keinen Problemen kommt
df_creator_id_duplicates_droped = df_creator_id.drop_duplicates(subset=["ID"])
df_creator_id_duplicates_droped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192896 entries, 0 to 3149
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   ID          192896 non-null  int64
 1   creator_id  192896 non-null  int64
dtypes: int64(2)
memory usage: 4.4 MB


In [14]:
# Fertiges Dataframe für den Merge im Notebook: exportieren
df_creator_id.to_csv(os.path.join(os.path.dirname(dirname(notebook_path)), "data/ks-project-edited-creator-id.csv"))

In [17]:
# Erste Betrachtung, wie viele Projekte einzelne Creator haben
df_creator_id.groupby("creator_id").count().sort_values("ID", ascending=False)

,ID
creator_id,
2104052526,60
1229765692,57
1655558466,55
1077465647,53
957058942,46
...,...
792838343,1
792812762,1
792793313,1
